In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv(r'E:\CampusX\DSML_2\ny-taxi-project\data\raw\train.csv')

In [3]:
n,p = data.shape

In [4]:
n

1458644

In [5]:
p

11

In [6]:
data.shape

(1458644, 11)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [8]:
data.sample(10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
202931,id1553827,2,2016-05-17 23:18:45,2016-05-17 23:44:50,1,-73.862732,40.768948,-73.980011,40.764706,N,1565
343281,id2937837,2,2016-01-11 15:12:54,2016-01-11 15:38:37,1,-73.864464,40.769753,-73.973656,40.754387,N,1543
887317,id0386399,1,2016-04-21 18:26:47,2016-04-21 18:50:53,3,-73.999748,40.726963,-73.984528,40.754646,N,1446
77851,id2560797,1,2016-06-25 10:27:33,2016-06-25 10:36:21,1,-73.992325,40.737911,-74.008133,40.745026,N,528
814128,id3222084,2,2016-04-30 11:07:03,2016-04-30 11:13:53,1,-73.990814,40.756092,-73.995148,40.741348,N,410
1314968,id3776579,2,2016-03-08 16:51:31,2016-03-08 17:06:03,1,-73.964523,40.801838,-73.940987,40.803337,N,872
917266,id3601542,2,2016-01-13 22:34:53,2016-01-13 22:48:57,1,-74.009842,40.720856,-73.987915,40.776249,N,844
1443335,id2951674,2,2016-02-11 23:11:54,2016-02-11 23:36:54,1,-73.862984,40.769341,-73.989471,40.757759,N,1500
1026766,id1419200,1,2016-01-07 15:55:55,2016-01-07 16:19:38,1,-73.997185,40.737129,-73.960754,40.772427,N,1423
1446376,id3211187,2,2016-04-30 02:56:05,2016-04-30 02:58:51,5,-73.978462,40.745522,-73.968750,40.758606,N,166


In [9]:
data.shape

(1458644, 11)

In [10]:
data.duplicated().sum()

0

In [11]:
data.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [12]:
data.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [13]:
data=data.drop(columns='id',axis=1)

In [14]:
data['vendor_id'].value_counts()

vendor_id
2    780302
1    678342
Name: count, dtype: int64

In [15]:
data['passenger_count'].value_counts()

passenger_count
1    1033540
2     210318
5      78088
3      59896
6      48333
4      28404
0         60
7          3
9          1
8          1
Name: count, dtype: int64

In [16]:
data['store_and_fwd_flag'].value_counts()

store_and_fwd_flag
N    1450599
Y       8045
Name: count, dtype: int64

In [17]:
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [18]:
data["pickup_hour"] = data['pickup_datetime'].dt.hour
data["pickup_minute"] = data['pickup_datetime'].dt.minute
data["pickup_second"] = data['pickup_datetime'].dt.second
data["pickup_day_week"] = data['pickup_datetime'].dt.dayofweek
data["pickup_month"] = data['pickup_datetime'].dt.month
data["pickup_minute_of_the_day"] = data["pickup_hour"]*60+data["pickup_minute"]

In [19]:
data["dropoff_hour"] = data['dropoff_datetime'].dt.hour
data["dropoff_minute"] = data['dropoff_datetime'].dt.minute
data["dropoff_second"] = data['dropoff_datetime'].dt.second
data["dropoff_day_week"] = data['dropoff_datetime'].dt.dayofweek
data["dropoff_month"] = data['dropoff_datetime'].dt.month
data["dropoff_minute_of_the_day"] = data["dropoff_hour"]*60+data["dropoff_minute"]

In [20]:
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,...,pickup_second,pickup_day_week,pickup_month,pickup_minute_of_the_day,dropoff_hour,dropoff_minute,dropoff_second,dropoff_day_week,dropoff_month,dropoff_minute_of_the_day
0,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,...,55,0,3,1044,17,32,30,0,3,1052
1,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,...,35,6,6,43,0,54,38,6,6,54
2,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,...,24,1,1,695,12,10,48,1,1,730
3,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,...,31,2,4,1172,19,39,40,2,4,1179
4,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,...,55,5,3,810,13,38,10,5,3,818


In [21]:
data.drop(columns=["pickup_datetime", "dropoff_datetime"], axis=1, inplace=True)

In [22]:
data["trip_duration"].value_counts()

trip_duration
368      1624
408      1584
348      1582
367      1581
358      1577
         ... 
7378        1
83250       1
6615        1
34014       1
6124        1
Name: count, Length: 7417, dtype: int64

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 20 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   vendor_id                  1458644 non-null  int64  
 1   passenger_count            1458644 non-null  int64  
 2   pickup_longitude           1458644 non-null  float64
 3   pickup_latitude            1458644 non-null  float64
 4   dropoff_longitude          1458644 non-null  float64
 5   dropoff_latitude           1458644 non-null  float64
 6   store_and_fwd_flag         1458644 non-null  object 
 7   trip_duration              1458644 non-null  int64  
 8   pickup_hour                1458644 non-null  int32  
 9   pickup_minute              1458644 non-null  int32  
 10  pickup_second              1458644 non-null  int32  
 11  pickup_day_week            1458644 non-null  int32  
 12  pickup_month               1458644 non-null  int32  
 13  pickup_minut

In [24]:
cat_columns = data.columns[data.dtypes=='object']
num_columns = data.columns[(data.dtypes=='float') | (data.dtypes=='int')]

In [25]:
num_columns

Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'pickup_hour', 'pickup_minute', 'pickup_second',
       'pickup_day_week', 'pickup_month', 'pickup_minute_of_the_day',
       'dropoff_hour', 'dropoff_minute', 'dropoff_second', 'dropoff_day_week',
       'dropoff_month', 'dropoff_minute_of_the_day'],
      dtype='object')

In [26]:
X= data.drop(columns='trip_duration',axis=1)
y = data.trip_duration

In [27]:
X.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_hour,pickup_minute,pickup_second,pickup_day_week,pickup_month,pickup_minute_of_the_day,dropoff_hour,dropoff_minute,dropoff_second,dropoff_day_week,dropoff_month,dropoff_minute_of_the_day
0,2,1,-73.982155,40.767937,-73.964630,40.765602,N,17,24,55,0,3,1044,17,32,30,0,3,1052
1,1,1,-73.980415,40.738564,-73.999481,40.731152,N,0,43,35,6,6,43,0,54,38,6,6,54
2,2,1,-73.979027,40.763939,-74.005333,40.710087,N,11,35,24,1,1,695,12,10,48,1,1,730
3,2,1,-74.010040,40.719971,-74.012268,40.706718,N,19,32,31,2,4,1172,19,39,40,2,4,1179
4,2,1,-73.973053,40.793209,-73.972923,40.782520,N,13,30,55,5,3,810,13,38,10,5,3,818


In [28]:
y.head()

0     455
1     663
2    2124
3     429
4     435
Name: trip_duration, dtype: int64

In [43]:
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1021050, 19), (437594, 19), (1021050,), (437594,))

In [44]:
preprocessor = ColumnTransformer(transformers=[
    ('ohe', OneHotEncoder(drop='first',dtype=np.int32),cat_columns),
    ('ss',StandardScaler(),num_columns)
], remainder='passthrough'
)

In [45]:
X_train= preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [46]:
preprocessor.get_feature_names_out()

array(['ohe__store_and_fwd_flag_Y', 'ss__pickup_longitude',
       'ss__pickup_latitude', 'ss__dropoff_longitude',
       'ss__dropoff_latitude', 'ss__pickup_hour', 'ss__pickup_minute',
       'ss__pickup_second', 'ss__pickup_day_week', 'ss__pickup_month',
       'ss__pickup_minute_of_the_day', 'ss__dropoff_hour',
       'ss__dropoff_minute', 'ss__dropoff_second', 'ss__dropoff_day_week',
       'ss__dropoff_month', 'ss__dropoff_minute_of_the_day',
       'remainder__vendor_id', 'remainder__passenger_count'], dtype=object)

In [47]:
def evaluate_model(true, predict):
    r2score = r2_score(true,predict)
    mse = mean_squared_error(true, predict)
    mae = mean_absolute_error(true, predict)
    rmse = np.sqrt(mse)

    return r2score, mse, mae, rmse

In [48]:
X_train.shape

(1021050, 19)

In [50]:
models = {
        'linearRegression': LinearRegression(),
        'ridge': Ridge(),
        'lasso': Lasso(),
        'elasticNet': ElasticNet(),
        'decisionTreeRegressor': DecisionTreeRegressor(),
        # 'randomForestRegressor': RandomForestRegressor(),
        # 'baggingRegressor': BaggingRegressor(),
        # 'adaBoostRegressor': AdaBoostRegressor(),
        # 'gradientBoostingRegressor': GradientBoostingRegressor()
}

In [52]:
params = {
        'linearRegression': {},
        'ridge': {
            'alpha': [1.0,0.5,1.5,0.8],
            'max_iter': [1000,800,900,1500]
        },
        'lasso': {
            'alpha': [1.0,0.5,1.5,0.8],
            'max_iter': [1000,800,900,1500]
        },
        'elasticNet': {
            'alpha': [1.0,0.5,1.5,0.8],
            'max_iter': [1000,800,900,1500],
            'l1_ratio': [0.5, 0.8,0.2]
        },
        'decisionTreeRegressor': {
            'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            'splitter': ['best', 'random'],
            'max_features': [0.5, 0.25]
        },
        'randomForestRegressor': {
            'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            'max_features': [0.5, 0.25],
            'n_estimators': [8,64,128,256,512]
        },
        'baggingRegressor': {
            "base_estimator": ["LinearRegression()", "Ridge()", "Lasso()", "DecisionTreeRegressor()"],
            "n_estimators": [500, 100, 250],
            "max_samples": [0.25, 0.50],
            "bootstrap": True,
            "max_features": [0.5, 0.25],
            "bootstrap_features": True,
            "random_state": 42
        },
        'adaBoostRegressor': {
            'learning_rate': [.1,.01,0.5,.001],
            'loss': ['linear','square','exponential'],
            'n_estimators': [8,64,128,256,512]
        },
        'gradientBoostingRegressor': {
            'loss': ['squared_error', 'absolute_error', 'huber', 'quantile'],
            'learning_rate': [.1,.01,0.5,.001],
            'n_estimators': [8,64,128,256,512],
            'subsample': [0.6,0.75,0.8,0.9],
            'max_features': [0.5, 0.25],
            'criterion': ['squared_error', 'friedman_mse']
        }

    }

In [56]:
report = {}
for i in range(len(list(models))):
        model = list(models.values())[i]
        param = params[list(models.keys())[i]]

        print('training....', model)
        gs = GridSearchCV(model, param, cv=5)
        gs.fit(X_train, y_train)

        model.set_params(**gs.best_params_)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_score_r2 = r2_score(y_train, y_train_pred)
        test_score_r2 = r2_score(y_test, y_test_pred)

        n, p = X_test.shape

        adj_r2 = 1 - (1 - test_score_r2) * (n - 1) / (n - p - 1)
        #print(train_score_r2, test_score_r2, adj_r2)

        report[list(models.keys())[i]] = {'r2_train': train_score_r2, 'r2_test': test_score_r2, 'adj_r2': adj_r2}
        print(report)

training.... LinearRegression()
{'linearRegression': {'r2_train': 0.009231708220534873, 'r2_test': 0.002918975672299151, 'adj_r2': 0.0028756811907664215}}
training.... Ridge(alpha=1.5, max_iter=1000)
{'linearRegression': {'r2_train': 0.009231708220534873, 'r2_test': 0.002918975672299151, 'adj_r2': 0.0028756811907664215}, 'ridge': {'r2_train': 0.009227643103983718, 'r2_test': 0.0035821208874600163, 'adj_r2': 0.00353885520050623}}
training.... Lasso()


c:\Users\rkgst\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.057e+13, tolerance: 3.365e+09
  model = cd_fast.enet_coordinate_descent(
c:\Users\rkgst\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.966e+12, tolerance: 2.843e+09
  model = cd_fast.enet_coordinate_descent(
c:\Users\rkgst\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.060e+13, toleranc

{'linearRegression': {'r2_train': 0.009231708220534873, 'r2_test': 0.002918975672299151, 'adj_r2': 0.0028756811907664215}, 'ridge': {'r2_train': 0.009227643103983718, 'r2_test': 0.0035821208874600163, 'adj_r2': 0.00353885520050623}, 'lasso': {'r2_train': 0.003845030583574771, 'r2_test': 0.007577342121959152, 'adj_r2': 0.007534249912413582}}
training.... ElasticNet()
{'linearRegression': {'r2_train': 0.009231708220534873, 'r2_test': 0.002918975672299151, 'adj_r2': 0.0028756811907664215}, 'ridge': {'r2_train': 0.009227643103983718, 'r2_test': 0.0035821208874600163, 'adj_r2': 0.00353885520050623}, 'lasso': {'r2_train': 0.003845030583574771, 'r2_test': 0.007577342121959152, 'adj_r2': 0.007534249912413582}, 'elasticNet': {'r2_train': 0.001756605064521466, 'r2_test': 0.004529043525074217, 'adj_r2': 0.004485818954663268}}
training.... DecisionTreeRegressor()
